# Model evaluation

In [ ]:
from fashion_classifier.config import model_dir, transform, target_transform
from fashion_classifier.data import test_data
from fashion_classifier.model  import NeuralNetwork

import torch
from torch import nn
from torch.utils.data import DataLoader

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
test_data = DataLoader(test_data(transform=transform, target_transform=target_transform))
model = torch.load(model_dir / 'trained_model.pt').to(device)

In [ ]:
def test_accuracy(dataloader, model):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    correct = 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

In [ ]:
test_accuracy(test_data, model)